In [ ]:
'''
This notebook contains the code to merge all PRICES data across the years based on sections.
There are datasets for 2012, 2013 and 2014

'''

In [ ]:
'''
The following code block will merge all "cover" files across the years.
* First, we are going to read the respective files and store them as data frames
* Next, we are going to define column mappings that I have already figured out via manual methods
'''

In [13]:
# This code block stores file paths to variables to make the code neat
# The stored variables are called in the read_excel function and stored as dataframes

import pandas as pd

# Store excel file locations to variables (change it as per your path to file)

path_2012=r"C:\Users\warra\Desktop\Freelance\data\data\PriceMerge\1. Merging by Parts\5. Section 2 Prices Of Consumption Items\2012_community-prices.csv"
path_2013=r"C:\Users\warra\Desktop\Freelance\data\data\PriceMerge\1. Merging by Parts\5. Section 2 Prices Of Consumption Items\2013_s2_prices.csv"
path_2014=r"C:\Users\warra\Desktop\Freelance\data\data\PriceMerge\1. Merging by Parts\5. Section 2 Prices Of Consumption Items\2014_s2_prices.csv"

# Read csv files
df_2012 = pd.read_csv(path_2012)
df_2013 = pd.read_csv(path_2013)
df_2014 = pd.read_csv(path_2014)


In [14]:
'''
This code block will standardize column names across the years to avoid discrepancies during the merging process.
For example, the cover data for 2013 is both the same as in 2014 data, but they have the same data under their differently named variables. Hence, we decide to rename such columns beforehand
We will add the updated name to the mapping dictionaries instead of the original names.
'''

# Rename columns in df_2012
df_2012.rename(columns={
    'ITEM_CODE': 'item_code',
    'S_1_Q_A': 's2_qa',
    'S_1_Q_B': 's2_qb',
    'DISTRICT_C': 's2_qc',
    'UC_D': 's2_qe',
    'MOUZA_E': 's2_qg',
    'S_1_Q_F': 's2_qi',
    'PRICE_ID': 'Price_ID',
    'community': 'Community',
    'District_kg': 'D_Kg',
    'UC_kg': 'UC_Kg',
    'Mouza_kg': 'M_Kg',
    'PROVINCE_ID': 'P_ID',
    'DISTRICT_ID': 'D_ID',
    'TEHSIL_ID': 'T_ID',
    'UNION_COUNCIL': 'UC_Name',
    'UC_ID': 'UC_ID',
    'MAUZA_ID': 'M_ID'
}, inplace=True)

# Rename columns in df_2013
df_2013.rename(columns={
    'itemno': 'item_code',
    'item': 'item_name',
    's1_qa': 's2_qa',
    's1_qb': 's2_qb',
    's1_qc': 's2_qc',
    's1_qe': 's2_qe',
    's1_qg': 's2_qg',
    's1_qi': 's2_qi'
}, inplace=True)

# df_2014 doesn't need renaming as it is the reference

In [15]:
# Updated mappings
mapping_2012 = [
    'cid', 'round', 'item_code', None, 's2_qa', 's2_qb', 's2_qc',
    None, 's2_qe', None, 's2_qg', None, 's2_qi', 'Price_ID', 'Community',
    'D_Kg', 'UC_Kg', 'M_Kg', 'P_ID', 'D_ID', 'T_ID', 'UC_Name',
    'UC_ID', 'M_ID'
]


mapping_2013 = [
    'cid', 'round', 'item_code', 'item_name', 's2_qa', 's2_qb', 's2_qc',
    None, 's2_qe', None, 's2_qg', None, 's2_qi', None, None, None,
    None, None, None, None, None, None, None
]

mapping_2014 = [
    'cid', 'round', 'item_code', 'item_name', 's2_qa', 's2_qb', 's2_qc',
    's2_qd', 's2_qe', 's2_qf', 's2_qg', 's2_qh', 's2_qi', None, None,
    None, None, None, None, None, None, None, None
]


In [16]:
# Create a list of all possible columns in the correct order
all_columns = []

for col in mapping_2012:
    if col and col not in all_columns:
        all_columns.append(col) 
   
for col in mapping_2013:
    if col and col not in all_columns:
        all_columns.append(col)
for col in mapping_2014:
    if col and col not in all_columns:
        all_columns.append(col)



In [17]:
import pandas as pd
import numpy as np

def standardize_and_merge(dfs, mappings, all_columns):
    merged_data = {col: [] for col in all_columns}

    for df, mapping in zip(dfs, mappings):
        print(f"Processing DataFrame with columns: {df.columns.tolist()}")
        for i, col in enumerate(mapping):
            if col:
                ref_col = col.strip()  # Remove leading/trailing whitespace
                if ref_col not in merged_data:
                    merged_data[ref_col] = []
                if ref_col in df.columns:
                    print(f"Appending data for column {ref_col}")
                    if isinstance(df[ref_col], pd.Series):
                        merged_data[ref_col].extend(df[ref_col].tolist())
                    elif isinstance(df[ref_col], pd.DataFrame):
                        print(f"Column {ref_col} is duplicated in DataFrame. Appending data for each duplicate.")
                        for _, series in df[ref_col].items():
                            merged_data[ref_col].extend(series.tolist())
                else:
                    print(f"Column {ref_col} not found in DataFrame. Adding NaNs.")
                    merged_data[ref_col].extend([np.nan] * len(df))
    
    max_len = max(len(v) for v in merged_data.values())
    for key in merged_data:
        col_len = len(merged_data[key])
        if col_len < max_len:
            merged_data[key].extend([np.nan] * (max_len - col_len))

    merged_df = pd.DataFrame.from_dict(merged_data)
    return merged_df


In [18]:
# Usage with dataframes and mappings
dfs = [ df_2012, df_2013, df_2014]
mappings = [ mapping_2012, mapping_2013, mapping_2014]

merged_df = standardize_and_merge(dfs, mappings, all_columns)

Processing DataFrame with columns: ['Unnamed: 0', 'round', 'Price_ID', 'cid', 'Community', 'P_ID', 'D_ID', 'T_ID', 'UC_Name', 'UC_ID', 'M_ID', 'item_code', 's2_qa', 's2_qb', 's2_qc', 's2_qe', 's2_qg', 's2_qi', 'D_Kg', 'UC_Kg', 'M_Kg']
Appending data for column cid
Appending data for column round
Appending data for column item_code
Appending data for column s2_qa
Appending data for column s2_qb
Appending data for column s2_qc
Appending data for column s2_qe
Appending data for column s2_qg
Appending data for column s2_qi
Appending data for column Price_ID
Appending data for column Community
Appending data for column D_Kg
Appending data for column UC_Kg
Appending data for column M_Kg
Appending data for column P_ID
Appending data for column D_ID
Appending data for column T_ID
Appending data for column UC_Name
Appending data for column UC_ID
Appending data for column M_ID
Processing DataFrame with columns: ['Unnamed: 0', 'cid', 'round', 'item_code', 'item_name', 's2_qa', 's2_qb', 's2_qc', '

In [19]:
# Rename columns for the merged file (if needed)
rename_mapping = {
    'cid': 'CID',
    'round': 'Survey_Round',
    'item_code': 'Item_Code',
    'item_name': 'Item_Name',
    's2_qa': 'Unit',
    's2_qb': 'Qty_Unit',
    's2_qc': 'D_Prices',
    's2_qd': 'D_ToS',
    's2_qe': 'UC_Prices',
    's2_qf': 'UC_ToS',
    's2_qg': 'M_Prices',
    's2_qh': 'M_ToS',
    's2_qi': 'Comments'
}

merged_df.rename(columns=rename_mapping, inplace=True)

# Save the merged dataframe to a CSV file
merged_df.to_csv('merged_Prices_Prices-Of-Consumption-Items.csv', index=False)
